In [2]:
%pip install -U -r requirements.txt


^C
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

  Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)


In [6]:
%pip install sentence-transformers


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\OMEN\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
os.environ["HF_TOKEN"] = getpass.getpass("Enter your HF API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain API key: ")

In [22]:
from langchain_groq import ChatGroq

# Replace this line
llm = ChatGroq(
    model="llama3-8b-8192",  # Updated model
    temperature=0,
    max_tokens=4000,
    timeout=None
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022084C91A00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000220837A7A00>, model_name='llama3-8b-8192', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=4000)

In [7]:
import csv
import json
import uuid
import cassio  # Using cassio for AstraDB connection
import pandas as pd

from langchain_huggingface import HuggingFaceEmbeddings

from langchain_groq import ChatGroq

# AstraDB connection details
ASTRA_DB_APPLICATION_TOKEN =  getpass.getpass("Enter your Astra DB Token: ")
ASTRA_DB_ID =  getpass.getpass("Enter your Astra DB ID: ")


# Initialize AstraDB connection with Cassio
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID
)

# Embeddings setup
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# ChatGroq LLM setup
# Replace this line
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Updated model
    temperature=0,
    max_tokens=4000,
    timeout=None
)


c:\Users\OMEN\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Entering data in the Vector DB for RAG

In [24]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
import cassio
import time
from cassandra import OperationTimedOut

# Create Cassandra vector store
astra_vector_store = Cassandra(
    embedding=embeddings,
    session=None,
    table_name="Projekt",
    keyspace=None
)

def process_csv(file_path, context_column, response_column):
    df = pd.read_csv(file_path)
    docs = []
    for _, row in df.iterrows():
        content = f"Context: {row[context_column]}\nResponse: {row[response_column]}"
        metadata = {"source": file_path, "type": "csv"}
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

def process_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    docs = []
    for item in data:
        content = f"Question: {item['question']}\nAnswer: {item['answer']}\nTopic: {item['topic']}"
        metadata = {"source": file_path, "type": "json", "topic": item['topic']}
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

# Process and add documents
docs = []
docs.extend(process_csv('amod.csv', 'Context', 'Response'))
docs.extend(process_json('cleaned_chats.json'))

print(f"Total documents to insert: {len(docs)}")

# === Insert in Batches ===
batch_size = 10

for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    try:
        astra_vector_store.add_documents(batch)
        print(f"Inserted batch {i // batch_size + 1}: {len(batch)} documents")
        time.sleep(0.5)  # Optional delay to avoid hitting API rate limits
    except OperationTimedOut as e:
        print(f"Timeout error on batch {i // batch_size + 1}: {e}")
    except Exception as e:
        print(f"Unexpected error on batch {i // batch_size + 1}: {e}")

# === Wrap Index ===
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

print("Vector index created successfully.")

Total documents to insert: 5641
Inserted batch 1: 10 documents
Inserted batch 2: 10 documents
Inserted batch 3: 10 documents
Inserted batch 4: 10 documents
Inserted batch 5: 10 documents
Inserted batch 6: 10 documents
Inserted batch 7: 10 documents
Inserted batch 8: 10 documents
Inserted batch 9: 10 documents
Inserted batch 10: 10 documents
Inserted batch 11: 10 documents
Inserted batch 12: 10 documents
Inserted batch 13: 10 documents
Inserted batch 14: 10 documents
Inserted batch 15: 10 documents
Inserted batch 16: 10 documents
Inserted batch 17: 10 documents
Inserted batch 18: 10 documents
Inserted batch 19: 10 documents
Inserted batch 20: 10 documents
Inserted batch 21: 10 documents
Inserted batch 22: 10 documents
Inserted batch 23: 10 documents
Inserted batch 24: 10 documents
Inserted batch 25: 10 documents
Inserted batch 26: 10 documents
Inserted batch 27: 10 documents
Inserted batch 28: 10 documents
Inserted batch 29: 10 documents
Inserted batch 30: 10 documents
Inserted batch 31

In [23]:
import datetime
import warnings
from IPython.display import display, HTML
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import logging


def format_message(role, content):
    """Formats the message with HTML for better display in Colab."""
    color = "#4a76a8" if role == "Bot" else "#6c757d"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"""
    <div style="margin-bottom: 10px; padding: 10px; border-radius: 5px; background-color: {color}; color: white;">
        <strong>{role}</strong> - {timestamp}<br>
        {content}
    </div>
    """


def chat_with_rag():
    # Memory setup (moved inside the function to ensure it's defined)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create the Conversational Retrieval Chain (RAG setup) (moved inside the function)
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=astra_vector_store.as_retriever(search_kwargs={"k": 3}),
        memory=memory
    )

    print("Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['exit', 'thankyou']:
            display(HTML(format_message("Bot", "Goodbye! Have a great day!")))
            break

        if not user_input:
            display(HTML(format_message("Bot", "Please enter a valid question or command.")))
            continue

        try:
            result = qa_chain.invoke({"question": user_input})
            display(HTML(format_message("Bot", result['answer'])))
        except Exception as e:
            error_message = f"Oops! Something went wrong. Please try again.\nError: {str(e)}"
            display(HTML(format_message("Bot", error_message)))

if __name__ == "__main__":
    chat_with_rag()


Hi, how are you feeling today?, You can type 'exit' or 'thankyou' to end the conversation.
